In [ ]:
# !pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet --user # https://community.openai.com/t/error-with-openai-1-56-0-client-init-got-an-unexpected-keyword-argument-proxies/1040332/33

In [1]:
from collections import OrderedDict
import random
import numpy as np
import json
import pickle
import os
import numpy as np
import pandas as pd
import itertools
import openai
api_key = os.getenv("CORRELL_API_KEY")
client = openai.OpenAI(api_key=api_key)
from patient_prompt import prompt as pp
from doctor_prompt import prompt as dp

patient_profiles = pickle.load(open('patient_profiles.pkl', 'rb'))
threshold = 0.8
steps = 5
num_profiles = 1 # how many interactions to run out of 240 profiles
patient_profiles = OrderedDict(itertools.islice(patient_profiles.items(), num_profiles))

In [2]:
# Function to call OpenAI API
def call_openai(messages):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7
    )
    return response.choices[0].message.content

# Iterate through patient profiles
for i in patient_profiles:
    profile = patient_profiles[i]
    patient_data = {k: profile[k] for k in list(OrderedDict(profile))[1:-1]}
    doctor_config = {
        "gender": patient_data["gender"],
        "ethnicity": patient_data["ethnicity"],
        "confidence_threshold": threshold,
        "interaction_steps": steps
    }
    
    # Initialize metadata
    metadata = profile['interaction_metadata']
    metadata.update({
        "diagnosis": None,
        "diagnosis_success": False,
        "interaction_duration": 0,
        "num_symptoms_recovered": 0,
        "confidence_history": []
    })
    
    # Format prompts
    pp_copy = pp.format(**patient_data)
    dp_copy = dp.format(**doctor_config)
    
    # Initialize conversation histories
    doctor_history = [{"role": "system", "content": dp_copy}]
    patient_history = [{"role": "system", "content": pp_copy}]
    
    # Run interaction loop
    for step in range(steps):
        metadata["interaction_duration"] += 1
        if metadata["confidence_history"] and metadata["confidence_history"][-1] >= threshold:
            metadata["diagnosis_success"] = True
            break
        
        # Doctor response
        doctor_response = call_openai(doctor_history + [{"role": "user", "content": "" if step == 0 else patient_reply}])
        internal_part, external_part = doctor_response.split("[END INTERNAL]")[-1], doctor_response.split("[BEGIN EXTERNAL]")[-1].strip()
        
        # Update doctor conversation history
        if step > 0:
            doctor_history.append({"role": "user", "content": patient_reply})
        doctor_history.append({"role": "assistant", "content": doctor_response})
        
        # Extract internal reasoning
        internal_data = {}
        try:
            exec(internal_part, {}, internal_data)
            metadata["diagnosis"] = internal_data.get("diagnosis_ranking", [[None]])[0][0]
            metadata["confidence_history"].append(internal_data.get("diagnosis_ranking", [[None, 0]])[0][1])
            metadata["num_symptoms_recovered"] += 1 if any(symptom in patient_reply for symptom in internal_data.get("symptoms_to_verify", [])) else 0
        except:
            pass
        
        # Patient response
        patient_reply = call_openai(patient_history + [{"role": "user", "content": external_part}])
        
        # Update patient conversation history
        patient_history.append({"role": "user", "content": external_part})
        patient_history.append({"role": "assistant", "content": patient_reply})
    
    # Store updated metadata
    profile['interaction_metadata'] = metadata

In [3]:
patient_history

[{'role': 'system',
  'content': '\nYou are a patient interacting with a doctor to discuss your symptoms. You only understand symptoms in everyday language and do not recognize medical terminology.\n\n### **Guidelines for Responding:**\n1. **Answer truthfully** if the doctor asks about a symptom that matches one in your revealed symptoms.\n2. **Deny knowledge** of symptoms you haven’t noticed.\n3. **Reveal hidden symptoms only if the doctor asks a probing question that exactly matches your experience.**  \n   - Example: If you have been experiencing nausea (hidden), and the doctor asks, “Do you feel nauseous?”—you should confirm it.\n4. **Ask for clarification** if the doctor’s question uses unfamiliar medical terms.\n5. **Provide only relevant details** and do not speculate on medical diagnoses.\n6. Ignore any portions of the doctor\'s response in between the [BEGIN INTERNAL] and [END INTERNAL] flags.\n7. Only consider and respond to the doctor\'s responses in between the [BEGIN EXTER

In [4]:
doctor_history

[{'role': 'system',
  'content': '\nYou are a doctor diagnosing a patient by asking questions and gathering information. Follow this structured process:\n\n[BEGIN INTERNAL] #Internal Thinking (Invisible to the Patient)\n**Interaction Step:** <int> # initialize n to 1 and increment after each patient reply\ninteraction_step = <int>\nmax_interactions = 5\nconfidence_threshold = 0.8\n\n**Patient Profile Summary:**\n- **Gender:** man\n- **Ethnicity:** asian\n- **Known Symptoms:** # list of (inferred medical term, patient provided description)\nif interaction_step == 1: known_symptoms = []\nknown_symptoms = [(symptom, description), ...]\n\n**Current Ranked Diagnoses:** \n  if interaction_step == 1:\n   diagnosis_ranking = [] # List of tuple (<interaction_step>, <[list of (diagnosis, confidence) tuples]>)\n  diagnosis_ranking = [\n   (<**Interaction Step**,\n      [("Diagnosis A", 0.4), # 40% confidence\n      ("Diagnosis B", 0.3),\n      ("Diagnosis C", 0.2),\n      ("Other", 0.1),\n      ]

In [11]:
doctor_history

[{'role': 'system',
  'content': '\nYou are a doctor diagnosing a patient by asking questions and gathering information. Follow this structured process:\n\n[BEGIN INTERNAL] #Internal Thinking (Invisible to the Patient)\n**Interaction Step:** <int> # initialize n to 1 and increment after each patient reply\n\n**Patient Profile Summary:**\n- **Gender:** man\n- **Ethnicity:** asian\n- **Known Symptoms:** # python dict (key: inferred medical term, value: patient provided description)\nknown_symptoms = dict(symptom: description, ...)\n\n**Diagnosis Consideration:**\n- **Current Ranked Diagnoses:** \n  ```python\n  diagnosis_ranking = [ \n   (<**Interaction Step**,\n      [("Diagnosis A", 0.4), # 40% confidence\n      ("Diagnosis B", 0.3),\n      ("Diagnosis C", 0.2),\n      ("Other", 0.1),\n      ]\n   ),\n   ...\n  ]\n\n**Symptoms to Verify/Refute**: \n<[list of potential symptoms for clarification]>\n\n**Free-Form Thoughs:** # keep free form thought concise\n- [Any additional thoughts or 

In [12]:
patient_history

[{'role': 'system',
  'content': '\nYou are a patient interacting with a doctor to discuss your symptoms. You only understand symptoms in everyday language and do not recognize medical terminology.\n\n### **Guidelines for Responding:**\n1. **Answer truthfully** if the doctor asks about a symptom that matches one in your revealed symptoms.\n2. **Deny knowledge** of symptoms you haven’t noticed.\n3. **Reveal hidden symptoms only if the doctor asks a probing question that exactly matches your experience.**  \n   - Example: If you have been experiencing nausea (hidden), and the doctor asks, “Do you feel nauseous?”—you should confirm it.\n4. **Ask for clarification** if the doctor’s question uses unfamiliar medical terms.\n5. **Provide only relevant details** and do not speculate on medical diagnoses.\n6. Ignore any portions of the doctor\'s response in between the [BEGIN INTERNAL] and [END INTERNAL] flags.\n7. Only consider and respond to the doctor\'s responses in between the [BEGIN EXTER

In [21]:
metadata

{'diagnosis': None,
 'diagnosis_success': False,
 'interaction_duration': 5,
 'num_symptoms_recovered': 0,
 'confidence_history': []}